In [1]:
from __future__ import print_function
import sys
# print (sys.version)
import argparse
import glob
import torchvision
import os
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from PIL import Image
from torchvision.transforms import ToTensor
#import tensorflow as tf 
import numpy as np

# from utils import (
#   read_data, 
#   input_setup, 
#   imsave,
#   merge,
#   modcrop)

import scipy.misc
import scipy.ndimage
import time
import matplotlib.pyplot as plt

import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
tf.disable_v2_behavior()
from PIL import Image  # for loading images as YCbCr format

import pylab
from pylab import * 
from matplotlib import *
import imageio
import h5py
import random
from skimage import transform,data
import cv2
#import tensorflow as tf

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
import sys
print(sys.executable)

c:\users\bmakh\anaconda3\envs\upvideo\python.exe


# 1- Manipulating my video : split into images and cobvert to bmp format

In [3]:

vidcap = cv2.VideoCapture('./farda1.mp4')
success,image = vidcap.read()
count = 0
while success:
    cv2.imwrite("./Dataset/Test/frame%d.bmp" % count, image)     # save frame as JPEG file   
    cv2.imwrite("./Dataset/JPEG/frame%d.jpg" % count, image)
    success,image = vidcap.read()
    #print('Read a new frame: ', success)
    count += 1

In [4]:
success

False

### Show images of the video 

In [ ]:
filename = './farda1.mp4'
vid = imageio.get_reader(filename,  'ffmpeg')
nums = [0, 1357]
#Le Nombre max est 1357
#x=0
#for num in nums:
    
 #   image = vid.get_data(num)
  #  fig = pylab.figure()
   # fig.suptitle('image #{}'.format(num), fontsize=20)
    #pylab.imshow(image)
for num in range(0, 50):
    image = vid.get_data(num)
    fig = pylab.figure()
    fig.suptitle('img #{}'.format(num), fontsize=20)
    pylab.imshow(image)
    #t.save('./dataset/'+'t'+num, 'bmp')
    pylab.savefig('./dataset/Test'+'test'+num, '.bmp')
#    x=x+num
#print(x)
pylab.show()

In [ ]:
%pwd

# 2- Defining the architecture of my model 

In [5]:
#############      Save the model with its check points    ################
def save(checkpoint_dir, step):
    model_name = "SRCNN.model"
    model_dir = "%s_%s" % ("srcnn", label_size)
    checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    saver.save(sess,
                    os.path.join(checkpoint_dir, model_name),
                    global_step=step)
    

In [6]:
######      Load the model   and its checkpoints ########
def load(checkpoint_dir):
    print(" [*] Reading checkpoints...")
    model_dir = "%s_%s" % ("srcnn", label_size)
    checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt and ckpt.model_checkpoint_path:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        saver.restore(sess, os.path.join(checkpoint_dir, ckpt_name))
        return True
    else:
        return False

In [7]:
epoch=1500  #Number of epoch [15000]
batch_size= 128 #The size of batch images [128]")
image_size= 33 #"The size of image to use [33]")
label_size= 21 #"The size of label to produce [21]")
learning_rate= 1e-4 #"The learning rate of gradient descent algorithm [1e-4]")
c_dim= 1 #"Dimension of image color. [1]")
scale= 3 #"The size of scale factor for preprocessing input image [3]")
stride= 21 #"The size of stride to apply input image [14]")
checkpoint_dir= "output/checkpoint" # "Name of checkpoint directory [checkpoint]")
sample_dir= "output/sample"# "Name of sample directory [sample]")
model= "srcnn" # "Which module to use [srcnn]")
is_train= True #"True for training, False for testing [True]")


# 3- Manipulating the data (images to build and train the model)

In [8]:
# check if it exist images or not (with the specified format)
def is_image_file(filename):
    return any(filename.endswith(extension) for extension in [".bmp", ".png", ".jpg", ".jpeg"])

In [9]:
def modcrop(image, scale=3): #
    """
      To scale down and up the original image, first thing to do is to have no remainder while scaling operation.

      We need to find modulo of height (and width) and scale factor.
      Then, subtract the modulo from height (and width) of original image size.
      There would be no remainder even after scaling operation.
      """
    if len(image.shape) == 3:
        h, w, _ = image.shape
        print(h)
        h = h - np.mod(h, scale)
        print(w)
        w = w - np.mod(w, scale)
        image = image[0:h, 0:w, :]
    else:
        h, w = image.shape
        h = h - np.mod(h, scale)
        w = w - np.mod(w, scale)
        image = image[0:h, 0:w]
    return image

In [10]:
def imread(path, is_grayscale=True):
#   """
#   Read image using its path.
#   Default value is gray-scale, and image is read by YCbCr format as the paper said.
#   """
  if is_grayscale:
        return imageio.imread(path, as_gray=True, pilmode='YCbCr').astype(np.float)#scipy.misc.imread(path, flatten=True, mode='YCbCr')
#     img = cv2.imread(path)
#     return img = img.flatten().astype(np.float)
  else:
        img = cv2.imread(path)  # Read input imgae
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert color format form BGR to RGB (OpenCV default is BGR).
        img = img.astype(np.float)  # Convert ot float
        return img

In [11]:
def preprocess(path, is_train, scale): # Args are 
  
  if is_train:
    image = imread(path, is_grayscale=False) #it was false
    image = image[:,:,0] 
  
    # Must be normalized
    image = image / 255.

    label_ = modcrop(image, scale)
    input_ = scipy.ndimage.interpolation.zoom(label_, (1./scale), prefilter=False)
    input_ = scipy.ndimage.interpolation.zoom(input_, (scale/1.), prefilter=False)

    return input_, label_

  else:
    image = imread(path, is_grayscale=False)
    # Must be normalized

    h, w, _ = image.shape

    #no color change using this resize method
    image = transform.resize(image, (h//scale, w//scale, 3))
    image = transform.resize(image, (h, w, 3))

    input_dir = os.path.join(os.getcwd(),sample_dir,os.path.split(path)[-1])
    scipy.misc.imsave(input_dir, image)

    image = image / 255.

    return image

In [12]:
def imsave(image, path):
    image = image * 255.
    return scipy.misc.imsave(path, image)

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h*size[0], w*size[1], 1))
    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image

    return img

In [13]:
def make_data(sess, data, label=None): #Make input data as h5 file format
  # Depending on 'is_train' (flag value), savepath would be changed.
  if is_train:
    savepath = os.path.join(os.getcwd(), 'output/checkpoint/train.h5')
    with h5py.File(savepath, 'w') as hf: 
        hf.create_dataset('data', data=data)
        hf.create_dataset('label', data=label) 

In [14]:
def prepare_data(sess, dataset): # prepare the data for testing and training (test data and train data)
    # args are dataset (test/ train)
    # For train dataset output data would be ['.../t1.bmp', '.../t2.bmp', ..., '.../t99.bmp']
    
    if is_train:
        data_dir = os.path.join(os.getcwd(), dataset)
        data = [os.path.join(data_dir, x) for x in os.listdir(data_dir) if is_image_file(x)]
    else:
        data_dir = os.path.join(os.getcwd(), dataset)
        data = [os.path.join(data_dir, x) for x in os.listdir(data_dir) if is_image_file(x)]

    return data

In [15]:
def input_setup(sess, is_train, image_size, label_size, scale, stride ):
    """
      Read image files and make their sub-images and saved them as a h5 file format.
       """
  # Load data path
    if is_train:
         data = prepare_data(sess, dataset="dataset/Train")
    else:
         data = prepare_data(sess, dataset="dataset/Test")
    padding = abs(image_size - label_size) / 2 # 6
    if is_train:
        sub_input_sequence = []
        sub_label_sequence = []
        for i in range(len(data)):
            input_, label_ = preprocess(data[i], is_train, scale) 
            if len(input_.shape) == 3:
                h, w, _ = input_.shape
            else:
                h, w = input_.shape
            for x in range(0, h-image_size+1, stride):
                for y in range(0, w-image_size+1, stride):
                        sub_input = input_[x:x+image_size, y:y+image_size] # [33 x 33]
                        sub_label = label_[x+int(padding):x+int(padding)+label_size, y+int(padding):y+int(padding)+label_size] # [21 x 21]
                        # Make channel value
                        sub_input = sub_input.reshape([image_size, image_size, 1])  
                        sub_label = sub_label.reshape([label_size, label_size, 1])
                        sub_input_sequence.append(sub_input)
                        sub_label_sequence.append(sub_label)

        arrdata = np.asarray(sub_input_sequence) # [?, 33, 33, 1]
        arrlabel = np.asarray(sub_label_sequence) # [?, 21, 21, 1]
        make_data(sess, arrdata, arrlabel)

    else:
        nx = np.zeros(len(data), dtype=np.int)
        ny = np.zeros(len(data), dtype=np.int)   
        arr = []

        for i in range(len(data)):
            image = preprocess(data[i], is_train, scale)
            input_ = image
            sub_input_sequence = [] 
            if len(input_.shape) == 3:
                h, w, _ = input_.shape
            else:
                h, w = input_.shape

            # Numbers of sub-images in height and width of image are needed to compute merge operation.
            for x in range(0, h-image_size+1, stride):
                nx[i] += 1; ny[i] = 0
                for y in range(0, w-image_size+1, stride):
                    ny[i] += 1
                    sub_input = input_[x:x+image_size, y:y+image_size, :] # [33 x 33]
                    sub_input = sub_input.reshape([image_size, image_size, 3])  
                    sub_input_sequence.append(sub_input)

            arr.append(np.asarray(sub_input_sequence))
            sub_input_sequence.clear()
#               """
#               len(sub_input_sequence) : the number of sub_input (33 x 33 x ch) in one image
#               (sub_input_sequence[0]).shape : (33, 33, 1)
#               """
    if not is_train:
        return nx, ny, np.asarray(arr)

In [16]:
# Read the data formed by label and normal data 
# the data is saved in h5 format 
def read_data(path,is_train): # args are path , data and label  
#     path: file path of desired file
#     data: '.h5' file format that contains train data values
#     label: '.h5' file format that contains train label values
  with h5py.File(path, 'r') as hf:
    data = np.array(hf.get('data'))
    if is_train:
        label = np.array(hf.get('label'))
        return data, label
    else:
        return data

### Start building the model 

In [17]:
# Build the model 
image_size=33
label_size=21
batch_size=128
c_dim=1
is_grayscale = (c_dim == 1)
#x = tf.compat.v1.placeholder(tf.float32, shape=(1024, 1024))
images = tf.placeholder(tf.float32, [None, image_size, image_size, c_dim], name='images')
labels = tf.placeholder(tf.float32, [None, label_size, label_size, c_dim], name='labels')

weights = {
  'w1': tf.Variable(tf.random_normal([9, 9, 1, 64], stddev=1e-3), name='w1'),
  'w2': tf.Variable(tf.random_normal([1, 1, 64, 32], stddev=1e-3), name='w2'),
  'w3': tf.Variable(tf.random_normal([5, 5, 32, 1], stddev=1e-3), name='w3')
}

biases = {
  'b1': tf.Variable(tf.zeros([64]), name='b1'),
  'b2': tf.Variable(tf.zeros([32]), name='b2'),
  'b3': tf.Variable(tf.zeros([1]), name='b3')
}
def model(): 
    conv1 =  tf.nn.relu( tf.nn.conv2d(images, weights['w1'], strides=[1,1,1,1], padding='VALID') + biases['b1'])
    conv2 =  tf.nn.relu( tf.nn.conv2d(conv1, weights['w2'], strides=[1,1,1,1], padding='VALID') + biases['b2'])
    conv3 =  tf.nn.conv2d(conv2, weights['w3'], strides=[1,1,1,1], padding='VALID') + biases['b3']
    return conv3

In [18]:
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)
    

In [20]:

init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    if is_train:
        input_setup(sess, is_train, image_size, label_size, scale, stride )
    else:
        nx, ny, arr = input_setup(sess, is_train, image_size, label_size, scale, stride ) 
        print(np.shape(arr))

    if is_train:     
        data_dir = os.path.join('./{}'.format(checkpoint_dir), "train.h5")
        train_data, train_label = read_data(data_dir,is_train=True)


    pred=model()

# Loss function (MSE)
    loss = tf.reduce_mean(tf.square(labels - pred))

    saver = tf.train.Saver()  

    # Stochastic gradient descent with the standard backpropagation
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    counter = 0
    start_time = time.time()

    if load(checkpoint_dir):
        print(" [*] Load SUCCESS")
    else:
        print(" [!] Load failed...")

    if is_train:
        print("Training...")
        for ep in range(epoch):
        # Run by batch images
            batch_idxs = len(train_data) // batch_size
            for idx in range(0, batch_idxs):
                batch_images = train_data[idx*batch_size : (idx+1)*batch_size]
                batch_labels = train_label[idx*batch_size : (idx+1)*batch_size]
                counter += 1
                _, err = sess.run([train_op, loss], feed_dict={images: batch_images, labels: batch_labels})
                if counter % 10 == 0:
                    print("Epoch: [%2d], step: [%2d], time: [%4.4f], loss: [%.8f]" % ((ep+1), counter, time.time()-start_time, err))
                if counter % 500 == 0:
                    save(checkpoint_dir, counter)

    else:
        print("Testing...")
        for i in range(len(arr)):
            image = np.zeros((nx[i]*stride,ny[i]*stride,3))
            for j in range(3):
                result = pred.eval({images: arr[i][:,:,:,j].reshape([nx[i]*ny[i], image_size, image_size, 1])})
                result = merge(result, [nx[i], ny[i]])
                result = result.squeeze()
                image[:, :, j] = result

            image_path = os.path.join(os.getcwd(), sample_dir)
            image_path = os.path.join(image_path, "test_output%03d.png"%i)
            imsave(image, image_path)

 [*] Reading checkpoints...
INFO:tensorflow:Restoring parameters from output/checkpoint\srcnn_21\SRCNN.model-10500
 [*] Load SUCCESS
Training...
Epoch: [ 1], step: [10], time: [2.8752], loss: [0.47708213]
Epoch: [ 1], step: [20], time: [5.4987], loss: [0.45070845]
Epoch: [ 1], step: [30], time: [8.1490], loss: [0.33618870]
Epoch: [ 1], step: [40], time: [10.8168], loss: [0.20626441]
Epoch: [ 1], step: [50], time: [13.4138], loss: [0.29573232]
Epoch: [ 1], step: [60], time: [16.0483], loss: [0.63185948]
Epoch: [ 1], step: [70], time: [18.6979], loss: [0.46155137]
Epoch: [ 2], step: [80], time: [21.4921], loss: [0.47090638]
Epoch: [ 2], step: [90], time: [24.1182], loss: [0.30255875]
Epoch: [ 2], step: [100], time: [26.7351], loss: [0.50271297]
Epoch: [ 2], step: [110], time: [29.4065], loss: [0.42514205]
Epoch: [ 2], step: [120], time: [32.0514], loss: [0.37401330]
Epoch: [ 2], step: [130], time: [34.7281], loss: [0.13774389]
Epoch: [ 2], step: [140], time: [37.3935], loss: [0.19046076]

KeyboardInterrupt: 